# Métricas de Avaliação - Dataset SVM KubeMon

Este notebook prepara os dados a serem utilizados para os modelos.


In [50]:
# Importações necessárias
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, LabelEncoder, PowerTransformer
from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score,
    precision_score, recall_score, f1_score, roc_auc_score,
    roc_curve, precision_recall_curve, auc
)
import warnings
warnings.filterwarnings('ignore')

# Configurações de visualização
plt.style.use('default')
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("📚 Bibliotecas importadas com sucesso!")
print("🎯 Pronto para avaliar métricas do modelo SVM")

📚 Bibliotecas importadas com sucesso!
🎯 Pronto para avaliar métricas do modelo SVM


In [51]:
#Variáveis globais
import pickle
import lib_analise 
info_modelo = lib_analise.get_info_modelo()  # para garantir que a função está carregada da

nome_dataset = info_modelo['nome_dataset']
arq_dataset_csv = info_modelo['parametros']['arq_dataset_csv']
arq_dataset_pkl = info_modelo['parametros']['arq_dataset_pkl']

#carregar os datasets de teste, treino e validação do arquivo pickle
with open(arq_dataset_pkl, 'rb') as f:
    datasets = pickle.load(f)


## 1. Carregamento e Preparação dos Dados

In [52]:
# 1) Carregamento e higienização (por dataset)

# Carregar o dataset
print("📂 Carregando dataset svm.csv...")

try:
    df = pd.read_csv(arq_dataset_csv)
    print(f"✅ Dataset carregado com sucesso!")
    print(f"📊 Dimensões: {df.shape[0]:,} linhas × {df.shape[1]} colunas")
    
except Exception as e:
    print(f"❌ Erro ao carregar dataset: {e}")
    print("🔄 Tentando carregar uma amostra...")
    try:
        df = pd.read_csv(arq_dataset_csv, nrows=10000)
        print(f"✅ Amostra carregada: {df.shape[0]:,} linhas")
    except:
        raise Exception("Não foi possível carregar o dataset")

# Verificar se a coluna target existe
if 'target' not in df.columns:
    print("❌ Coluna 'target' não encontrada!")
    print(f"Colunas disponíveis: {list(df.columns)}")
    raise Exception("Coluna target não encontrada")

print(f"\n📋 Informações do Target:")
target_counts = df['target'].value_counts()
print(f"   • Classes: {dict(target_counts)}")
for classe, count in target_counts.items():
    percentage = (count / len(df)) * 100
    print(f"   • {classe}: {count:,} ({percentage:.1f}%)")

# Mostrar informações gerais
print(f"\n📊 Informações Gerais:")
print(f"   • Valores ausentes no target: {df['target'].isnull().sum()}")
print(f"   • Total de features: {df.shape[1] - 1}")
print(f"   • Balanceamento: {'Balanceado' if target_counts.min() / target_counts.max() > 0.8 else 'Desbalanceado'}")

📂 Carregando dataset svm.csv...
✅ Dataset carregado com sucesso!
📊 Dimensões: 80,648 linhas × 126 colunas

📋 Informações do Target:
   • Classes: {'interf': np.int64(45660), 'normal': np.int64(34988)}
   • interf: 45,660 (56.6%)
   • normal: 34,988 (43.4%)

📊 Informações Gerais:
   • Valores ausentes no target: 0
   • Total de features: 125
   • Balanceamento: Desbalanceado


In [53]:
# Preparação dos dados
print("🔧 Preparando dados para treinamento...")

# Separar features e target
colunas_excluir = ['os_timestamp', 'node_name', 'iteration', 'target']
colunas_excluir = [col for col in colunas_excluir if col in df.columns]

# Selecionar apenas features numéricas
features_numericas = df.select_dtypes(include=[np.number]).columns.tolist()
features_para_modelo = [col for col in features_numericas if col not in colunas_excluir]

print(f"📊 Features para o modelo:")
print(f"   • Total de features: {len(features_para_modelo)}")
print(f"   • Colunas excluídas: {colunas_excluir}")

# Preparar X e y
X = df[features_para_modelo].copy()
y = df['target'].copy()

# Tratar valores ausentes
valores_ausentes = X.isnull().sum().sum()
if valores_ausentes > 0:
    print(f"   • Preenchendo {valores_ausentes:,} valores ausentes com a mediana...")
    X = X.fillna(X.median())

# Codificar target se necessário
le = LabelEncoder()
if y.dtype == 'object':
    y_encoded = le.fit_transform(y)
    classes_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    print(f"   • Target codificado: {classes_mapping}")
else:
    y_encoded = y.values
    classes_mapping = None

print(f"\n✅ Dados preparados:")
print(f"   • Shape X: {X.shape}")
print(f"   • Shape y: {len(y_encoded)}")
print(f"   • Classes únicas: {np.unique(y_encoded)}")

🔧 Preparando dados para treinamento...
📊 Features para o modelo:
   • Total de features: 122
   • Colunas excluídas: ['os_timestamp', 'node_name', 'iteration', 'target']
   • Target codificado: {'interf': np.int64(0), 'normal': np.int64(1)}

✅ Dados preparados:
   • Shape X: (80648, 122)
   • Shape y: 80648
   • Classes únicas: [0 1]


## 2. Divisão dos Dados e Normalização
# Utilizando Yeo-Johnson

In [54]:
# 3) Particionamento estratificado 40/30/30

# Divisão treino/teste
print("📊 Dividindo dados em treino e teste...")

# Dividindo o teste mantendo a proporção das classes (stratify) em 40% treino e 60% teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, 
    test_size=0.6, 
    random_state=42, 
    stratify=y_encoded
)

# Dividindo o teste em teste e validação (50%/50%)
X_test, X_val, y_test, y_val = train_test_split(
    X_test, y_test, 
    test_size=0.5, 
    random_state=42, 
    stratify=y_test
)

print(f"📊 Divisão dos dados:")
print(f"   • Treino: {X_train.shape[0]:,} amostras ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"   • Teste:  {X_test.shape[0]:,} amostras ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"   • Validação: {X_val.shape[0]:,} amostras ({X_val.shape[0]/len(X)*100:.1f}%)")

# Verificar distribuição das classes
print(f"\n📋 Distribuição das classes:")
unique_train, counts_train = np.unique(y_train, return_counts=True)
unique_test, counts_test = np.unique(y_test, return_counts=True)
unique_val, counts_val = np.unique(y_val, return_counts=True)

for classe in unique_train:
    train_pct = (counts_train[unique_train == classe][0] / len(y_train)) * 100
    test_pct = (counts_test[unique_test == classe][0] / len(y_test)) * 100
    val_pct = (counts_val[unique_val == classe][0] / len(y_val)) * 100
    print(f"   • Classe {classe}: Treino {train_pct:.1f}% | Teste {test_pct:.1f}% | Validação {val_pct:.1f}%")

# Normalização dos dados usando Yeo-Johnson
print(f"\n⚖️ Aplicando transformação Yeo-Johnson...")
yeo_johnson_transformer = PowerTransformer(method='yeo-johnson', standardize=True)
# O fit é feito apenas no conjunto de treino para evitar data leakage
X_train_scaled = yeo_johnson_transformer.fit_transform(X_train)
X_test_scaled = yeo_johnson_transformer.transform(X_test)
X_val_scaled = yeo_johnson_transformer.transform(X_val)

print(f"   ✅ Transformação Yeo-Johnson aplicada com StandardScaler integrado")
print(f"   • Média treino antes: {X_train.mean().mean():.3f} | depois: {X_train_scaled.mean():.3f}")
print(f"   • Std treino antes: {X_train.std().mean():.3f} | depois: {X_train_scaled.std().mean():.3f}")
print(f"   • Média teste antes: {X_test.mean().mean():.3f} | depois: {X_test_scaled.mean():.3f}")
print(f"   • Std teste antes: {X_test.std().mean():.3f} | depois: {X_test_scaled.std().mean():.3f}")
print(f"   • Média validação antes: {X_val.mean().mean():.3f} | depois: {X_val_scaled.mean():.3f}")
print(f"   • Std validação antes: {X_val.std().mean():.3f} | depois: {X_val_scaled.std().mean():.3f}")

print(f"   • Transformação aplicada: Yeo-Johnson + Padronização")


📊 Dividindo dados em treino e teste...
📊 Divisão dos dados:
   • Treino: 32,259 amostras (40.0%)
   • Teste:  24,194 amostras (30.0%)
   • Validação: 24,195 amostras (30.0%)

📋 Distribuição das classes:
   • Classe 0: Treino 56.6% | Teste 56.6% | Validação 56.6%
   • Classe 1: Treino 43.4% | Teste 43.4% | Validação 43.4%

⚖️ Aplicando transformação Yeo-Johnson...
   ✅ Transformação Yeo-Johnson aplicada com StandardScaler integrado
   • Média treino antes: 970513.048 | depois: -0.000
   • Std treino antes: 4262885.915 | depois: 0.805
   • Média teste antes: 986004.100 | depois: -0.000
   • Std teste antes: 4481052.608 | depois: 1.453
   • Média validação antes: 982812.774 | depois: -0.001
   • Std validação antes: 4428113.690 | depois: 0.793
   • Transformação aplicada: Yeo-Johnson + Padronização


In [55]:
## 3. Gera pickles com os datasets e grava um arquivo
# Salvar datasets em arquivos pickle gera um único arquivo para todos datasets
import pickle  

with open(arq_dataset_pkl, 'wb') as f:
    pickle.dump({
        'X_train': X_train,
        'X_test': X_test,
        'X_val': X_val,
        'y_train': y_train,
        'y_test': y_test,
        'y_val': y_val,
        'X_train_scaled': X_train_scaled,
        'X_test_scaled': X_test_scaled,
        'X_val_scaled': X_val_scaled,
        'classes_mapping': classes_mapping, 
        'features_ganho_informacao': features_para_modelo
    }, f)

print(f"✅ Datasets salvos em arquivos pickle com sucesso!")

✅ Datasets salvos em arquivos pickle com sucesso!


In [56]:
#carregar os datasets de teste, treino e validação do arquivo pickle
with open(arq_dataset_pkl, 'rb') as f:
    datasets = pickle.load(f)

X_train = datasets['X_train']
X_test = datasets['X_test']
X_val = datasets['X_val']
y_train = datasets['y_train']
y_test = datasets['y_test']
y_val = datasets['y_val']
X_train_scaled = datasets['X_train_scaled']
X_test_scaled = datasets['X_test_scaled']
X_val_scaled = datasets['X_val_scaled']
classes_mapping = datasets['classes_mapping']
features_ganho_informacao  = datasets['features_ganho_informacao']


